In [ ]:
# Install necessary libraries
!pip install pydicom scikit-image matplotlib numpy

import pydicom
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import os
from skimage.metrics import structural_similarity as ssim

def extract_zip(zip_path, extract_to_folder):
    """Extract the contents of the ZIP file."""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)

def find_dicom_files(folder_path):
    """Recursively search for DICOM files."""
    dicom_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.dcm'):
                dicom_files.append(os.path.join(root, file))
    return dicom_files

def load_dicom_images(folder_path):
    """Find all DICOM files in the folder and subdirectories and load them."""
    dicom_files = find_dicom_files(folder_path)
    dicom_images = []

    print(f"Found DICOM files: {dicom_files}")

    if not dicom_files:
        raise FileNotFoundError("No DICOM files found.")

    for dicom_file in dicom_files:
        try:
            ds = pydicom.dcmread(dicom_file)
            image_array = ds.pixel_array

            # Debug: Print the shape and details of the image array
            print(f"Loaded image shape from {dicom_file}: {image_array.shape}")
            print(f"Image metadata: {ds}")

            # Ensure image_array is 2D
            if image_array.ndim == 1:
                # Assume 1D array needs reshaping
                size = int(np.sqrt(image_array.size))
                if size * size == image_array.size:
                    image_array = image_array.reshape((size, size))
                else:
                    print(f"Skipping {dicom_file} due to incorrect 1D shape.")
                    continue
            elif image_array.ndim != 2:
                print(f"Skipping {dicom_file} due to unexpected shape: {image_array.shape}")
                continue

            dicom_images.append(image_array)
        except Exception as e:
            print(f"Error reading {dicom_file}: {e}")

    if not dicom_images:
        raise FileNotFoundError("No valid DICOM images found or all DICOM files are empty.")

    # Check the shapes of all images
    shapes = [img.shape for img in dicom_images]
    print(f"Shapes of loaded images: {shapes}")

    # Ensure all images have the same shape
    if len(set(shapes)) != 1:
        raise ValueError("DICOM images have different shapes and cannot be stacked.")

    return np.stack(dicom_images)

def visualize_slice(img_data, slice_index):
    """Visualize a specific slice of the image data."""
    if slice_index < 0 or slice_index >= img_data.shape[0]:
        raise ValueError("Slice index out of bounds.")

    slice_data = img_data[slice_index]

    # Debug: Check the shape of the slice data
    print(f"Shape of slice data for index {slice_index}: {slice_data.shape}")

    if slice_data.ndim == 1:
        size = int(np.sqrt(slice_data.size))
        if size * size == slice_data.size:
            slice_data = slice_data.reshape((size, size))
        else:
            raise ValueError("Cannot reshape slice data into 2D.")
    elif slice_data.ndim != 2:
        raise ValueError("Slice data must be 2D.")

    plt.imshow(slice_data, cmap="gray", origin="lower")
    plt.title(f"Slice {slice_index}")
    plt.axis("off")
    plt.show()

def analyze_images(img_data):
    """Compute basic statistics and plot histograms of pixel intensities."""
    mean = np.mean(img_data)
    median = np.median(img_data)
    std_dev = np.std(img_data)

    print(f"Image statistics:\nMean: {mean}\nMedian: {median}\nStandard Deviation: {std_dev}")

    plt.figure(figsize=(12, 6))
    plt.hist(img_data.ravel(), bins=100, color='gray', edgecolor='black')
    plt.title("Histogram of Pixel Intensities")
    plt.xlabel("Pixel Intensity")
    plt.ylabel("Frequency")
    plt.show()

def compare_images(img_data1, img_data2, slice_index):
    """Compare slices from two sets of images using SSIM."""
    if img_data1.shape != img_data2.shape:
        raise ValueError("Images must have the same shape for comparison.")

    slice_data1 = img_data1[slice_index]
    slice_data2 = img_data2[slice_index]

    print(f"Shape of slice_data1 for index {slice_index}: {slice_data1.shape}")
    print(f"Shape of slice_data2 for index {slice_index}: {slice_data2.shape}")

    if slice_data1.ndim == 1:
        size = int(np.sqrt(slice_data1.size))
        if size * size == slice_data1.size:
            slice_data1 = slice_data1.reshape((size, size))
        else:
            raise ValueError("Cannot reshape slice_data1 into 2D.")

    if slice_data2.ndim == 1:
        size = int(np.sqrt(slice_data2.size))
        if size * size == slice_data2.size:
            slice_data2 = slice_data2.reshape((size, size))
        else:
            raise ValueError("Cannot reshape slice_data2 into 2D.")

    if slice_data1.ndim != 2 or slice_data2.ndim != 2:
        raise ValueError("Slices must be 2D arrays.")

    slice_data1 = slice_data1.astype(np.float64)
    slice_data2 = slice_data2.astype(np.float64)
    ssim_value, _ = ssim(slice_data1, slice_data2, full=True, data_range=slice_data2.max() - slice_data2.min())
    print(f"SSIM between slices at index {slice_index}: {ssim_value:.4f}")

    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(slice_data1, cmap="gray", origin="lower")
    ax[0].set_title("Image 1 Slice")
    ax[0].axis("off")

    ax[1].imshow(slice_data2, cmap="gray", origin="lower")
    ax[1].set_title("Image 2 Slice")
    ax[1].axis("off")

    plt.show()

def create_montage(img_data, num_cols=5):
    """Create and plot a montage view of the image slices."""
    num_slices = img_data.shape[0]
    num_rows = (num_slices + num_cols - 1) // num_cols  # Calculate number of rows needed

    # Create a blank canvas for the montage
    montage = np.zeros((num_rows * img_data.shape[1], num_cols * img_data.shape[2]), dtype=img_data.dtype)

    for i in range(num_slices):
        row = i // num_cols
        col = i % num_cols
        montage[row * img_data.shape[1]:(row + 1) * img_data.shape[1],
                col * img_data.shape[2]:(col + 1) * img_data.shape[2]] = img_data[i]

    # Plot the montage
    plt.figure(figsize=(12, 12))
    plt.imshow(montage, cmap='gray', origin='lower')
    plt.title("Montage View of Image Slices")
    plt.axis('off')
    plt.show()

def main(zip_path, extract_to_folder):
    """Main function to run the processing pipeline."""
    extract_zip(zip_path, extract_to_folder)

    img_data = load_dicom_images(extract_to_folder)

    if img_data.size == 0:
        raise FileNotFoundError("No valid DICOM files found in the extracted folder.")

    central_slice = img_data.shape[0] // 2
    visualize_slice(img_data, central_slice)

    create_montage(img_data)

    analyze_images(img_data)

    if len(img_data) > 1:
        compare_images(img_data[0], img_data[1], central_slice)

    for root, _, files in os.walk(extract_to_folder, topdown=False):
        for file in files:
            os.remove(os.path.join(root, file))
        os.rmdir(root)

# Paths in Colab
zip_path = '/content/MRI data.zip'  # Adjust this if the ZIP file has a different name or path
extract_to_folder = '/content/extracted_dicom_data'

# Create the extraction folder if it doesn't exist
os.makedirs(extract_to_folder, exist_ok=True)

# Execute the main function
main(zip_path, extract_to_folder)
